In [ ]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install torchvision

In [9]:

from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as opti
import csv


def wordextractor(sentence, number):
    words=sentence.split(" ")
    return words[number]

def casetonumber(case):
    switch = {
        "Nom": 1,
        "Acc": 2,
        "Ins": 3,
        "Ine": 4, 
        "Sup": 5,
        "Sub": 6
    }
    return switch.get(case, "Invalid case")

def BERT(sentence, tokenizer, model):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors = "pt")
    outputs = model(**inputs)
    return outputs

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

s=["Elmegyek a boltba paradicsomért!", "Nem szeiojretem a szilvát."]
print(BERT(s, tokenizer, model)[0].size())

AttributeError: 'Tensor' object has no attribute 'Size'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

file1 = open("train.tsv")
train_tsv=csv.reader( file1, delimiter="\t")

file2 = open("test.tsv")
test_tsv=csv.reader( file2, delimiter="\t")

training_data=[]
test_data=[]
i=0 
for obj in train_tsv:
    training_data.append([])
    training_data[i].append(obj[0])
    training_data[i].append(obj[2])
    training_data[i].append(casetonumber(obj[3]))
    i=i+1

i=0
for obj in test_tsv:
    test_data.append([])
    test_data[i].append(obj[0])
    test_data[i].append(obj[2])
    test_data[i].append(casetonumber(obj[3]))
    i=i+1

trainloader=torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader=torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)
file1.close()
file2.close()
print("DONE")


In [5]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, X):
        h = self.input_layer(X)
        h = self.relu(h)
        out = self.output_layer(h)
        return out